In [3]:
#given a list of demand locations and time windows and number of available drivers and time restrictions of drivers,
#return the path of each driver, up to the point of return to the depot;  
#all demands fulfilled will be removed; those unfulfilled are carried over to the next decision epoch; 
#all drivers will remain in the system but their exit time will remain fixed so they cannot be assigned an order past that time
import numpy as np

import gurobipy as gp
from gurobipy import GRB
from gurobipy import quicksum
 
class color:
    PURPLE = '\033[95m'
    CYAN = '\033[96m'
    DARKCYAN = '\033[36m'
    BLUE = '\033[94m'
    GREEN = '\033[92m'
    YELLOW = '\033[93m'
    RED = '\033[91m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'
    END = '\033[0m'

def decode(x_hat): #decoder; algorithm 2 page 33 
    N = len(x_hat)
    x_hat = [0]+x_hat
    #print('x_hat: ', x_hat)
    i = 1; x = []
    while i <= 4:
        #print('i: ', i) 
        delta = 2**(4-i)
        j = 1

        while j <= N-delta+1:
            #print('j: ', j) 
            ind = 1
            k = j 
            while k <= j+delta -1:
                ind=ind*x_hat[k]
                #print('k: ', k) 
                k+=1 

            if ind > 0:
                x.append([j, j+delta-1])
                k=j

                while k <= j+delta -1:
                    x_hat[k]-=1
                    #print('k: ', k) 
                    k+=1
            else:
                j+=1
        i+=1 
    return x 

def find_time_visited(loc, b, driver_time_end, driver_time_start, fulfilled): 
    
    #loc = demand locations, b = latest time each demand can be fulfilled
    #driver_time_end = time by which driver must return to the depot  
    #returns time_visited a dictionary with keys [i,k], i = the index from 0 to len(loc) indicating the node visited, 
    #and k is the vehicle, so vehicle/driver k visited node i at time_visited[(i,k)] 
    
    V = len(driver_time_end) 
 
    def t(i,j):
        if j == len(loc): #0 and len(loc) both represent the depot 
            j = 0 
        return ((loc[i][0]-loc[j][0])**2 + (loc[i][1]-loc[j][1])**2)**0.5 

    N = len(loc); 

    m = gp.Model('vehicle_routing')
    
    m.Params.LogToConsole = 0
    
    x = m.addVars([(i,j,k) for i in range(N) for j in range(1,N+1) for k in range(V)], name = 'x', vtype = GRB.BINARY)

    m.addConstrs(quicksum([x[i,j,k] for j in range(1,N+1) for k in range(V) if i!=j]) <= 1 for i in range(1,N)) #CHANGED FROM == 1

    m.addConstrs(quicksum([x[i,i,k] for k in range(V)]) == 0 for i in range(1,N)) #cannot visit same node 

    m.addConstrs(quicksum([x[0,j,k] for j in range(1,N)]) <= 1 for k in range(V)); #CHANGED FROM == 1

    #m.addConstrs(quicksum([x[i,h,k] for h in range(1, N) for i in range(N)])==quicksum([x[h,j,k] for h in range(1, N) for j in range(1,N)]) for k in range(V)) 

    for h in range(1, N): 
        m.addConstrs(quicksum([x[i,h,k] for i in range(N)])==quicksum([x[h,j,k] for j in range(1,N+1)]) for k in range(V)) 

    m.update()

    m.addConstrs(quicksum([x[i,N,k] for i in range(N)]) <= 1 for k in range(V)) #CHANGED FROM == 1

    s = m.addVars([(i,k) for i in range(N+1) for k in range(V)], name = 's', vtype = GRB.CONTINUOUS)

    #m.addConstrs(s[0,k] == 0 for k in range(V)) #Eliminate this for more general case, and add the following constraint:  

    m.addConstrs(s[0,k] == driver_time_start[k] for k in range(V)) 

    M = 10e+4

    m.addConstrs(s[i,k] + t(i,j) - M*(1-x[i,j,k]) <= s[j,k] for i in range(N) for j in range(1,N+1) for k in range(V))  

    #m.addConstrs(s[i,k] + t(i,j) == s[j,k] for i in range(N) for j in range(1,N) for k in range(V) if x[i,j,k]==1) #nonlinear, incompatible 

    m.addConstrs(s[i,k] <= b[i] for i in range(1,N) for k in range(V)) 

    m.addConstrs(s[N,k] <= driver_time_end[k] for k in range(V)) #time driver k visits N is at most the exit time 
    
    numVisits = m.addVars([j for j in range(1,N)], name = 'numVisits') #number of times each customer is visited; numVisits <= 1 (it can be 0 unlike in classical VRP)

    m.addConstrs(numVisits[j] == quicksum([x.sum(i,j,'*') for i in range(N)]) for j in range(1,N)) 
    
    m.addConstrs(numVisits[j] <= (1-fulfilled[j]) for j in range(1,N)) #if already fulfilled (that is, if fulfilled = 1) cannot visit again 
    
    m.setObjective(quicksum([numVisits[j] for j in range(1, N)]), GRB.MAXIMIZE) 

    m.optimize()
    
    print('\nnumVisits: ', numVisits)
    visitedNodes = [j for j in range(1,N) if numVisits[j].x > 0]
    print('\nnodes visited indices: ', visitedNodes)
    print('\nObjective value: ', m.objval, ' Number of nodes visited: ', len(visitedNodes)) 
    
    nodes_visited = [] #nodes_visited[k] is the path of the kth driver 

    for k in range(V):
        nodes_visited.append([elt for elt in x if x[elt].x > 0 and elt[-1] == k]) 
    
    print('\nnodes_visited by each driver: ', nodes_visited)
    
    def find_index(index, arr): 
        for i, t in enumerate(arr):
            if t[0] == index:
                return i 
        return None 

    time_visited = {} 
    #s[i,k] is not the actual time driver k visits node i but s[i,k] >= actual time based on the constraint;
    #time_visited[(i,k)] is the actual time, found found the list of locations for each vehicle in nodes_visited 

    for k in range(V):
        time_visited[(0,k)]=driver_time_start[k] 

    for k in range(V):
        prev_node = 0  
        if nodes_visited[k] == []: #no node visited 
            continue 
        st = find_index(prev_node, nodes_visited[k])
        curr_node = nodes_visited[k][st][1]; 

        while st != None:  
            time_visited[(curr_node,k)] = t(prev_node, curr_node)+time_visited[(prev_node,k)]
            prev_node = curr_node; curr_node = nodes_visited[k][st][1]; 
            st = find_index(curr_node, nodes_visited[k])

        time_visited[(len(loc),k)] = time_visited[(prev_node,k)] + t(prev_node, len(loc))
    
    print('\ntime_visited: ', time_visited)
     
    for k in range(V):
        if (len(loc),k) in time_visited.keys(): 
            driver_time_start[k] = time_visited[(len(loc),k)] 
        else:
            driver_time_start[k] = time_visited[(0,k)] 
     
    for j in range(1,N):
        if numVisits[j].x > 0:
            fulfilled[j] = 1 
    
    print('\nupdated driver_time_start: ', driver_time_start); print('\nupdated fulfilled: ', fulfilled)
    
    return time_visited, fulfilled, driver_time_start    
   
x_hat = [1, 2, 5, 4, 4, 3, 2, 1] #number of company drivers available in each decision epoch 

x = decode(x_hat) #company driver shift schedules 

print('x: ', x)
 
num_epochs = len(x) 
  
#loc: locations of the demands (loc[0] = location of depot):

loc = [[0,0], [0.5,1], [0.1,0.2], [1,0.2]] 

loc = [[0,0], [5,10], [1,2], [10,2], [2,15], [10,5], [1,8], [9, 15], [4,7], [8,6], [1,5], [2, 3], [7, 1]] 

loc = [[0,0], [0.5,1], [0.1,0.2], [1,0.2], [5,4], [1,3], [2,7]] 

fulfilled = [0 for i in range(len(loc))] #0 = False, not fulfilled; irrelevant for the depot, fulfilled[0]

b = [float('inf')] + [100 for i in range(len(loc)-1)] 
#b is the latest time a node can be visited (the upper limit of the customer's time window); inf for depot
 
crowdsourced_driver_time_end = [4,7, 8, 5] #time by which each crowdsourced driver must return to depot; uncertain for crowdsourced drivers, company drivers can return at the latest epoch or later 
       
driver_time_end = crowdsourced_driver_time_end+[elt[1] for elt in x]
 
crowdsourced_driver_time_start = [1, 3, 5, 2] #time each driver is available

driver_time_start = crowdsourced_driver_time_start +[elt[0] for elt in x] 

print('driver_time_start: ', driver_time_start, ' driver_time_end: ', driver_time_end)

for t in range(1, num_epochs+1):
    #break 
    print('\n', color.BOLD, 't: ', t, color.END); print('\ninitial driver_time_start: ', driver_time_start)

    print('\ninitial fulfilled: ', fulfilled)
     
    time_visited, fulfilled, driver_time_start = find_time_visited(loc, b, driver_time_end, driver_time_start, fulfilled) 
    
    


x:  [[1, 8], [2, 5], [3, 6], [3, 6], [3, 3], [7, 7]]
driver_time_start:  [1, 3, 5, 2, 1, 2, 3, 3, 3, 7]  driver_time_end:  [4, 7, 8, 5, 8, 5, 6, 6, 3, 7]

  t:  1 

initial driver_time_start:  [1, 3, 5, 2, 1, 2, 3, 3, 3, 7]

initial fulfilled:  [0, 0, 0, 0, 0, 0, 0]

numVisits:  {1: <gurobi.Var numVisits[1] (value 1.0)>, 2: <gurobi.Var numVisits[2] (value 1.0)>, 3: <gurobi.Var numVisits[3] (value 1.0)>, 4: <gurobi.Var numVisits[4] (value 0.0)>, 5: <gurobi.Var numVisits[5] (value 1.0)>, 6: <gurobi.Var numVisits[6] (value 0.0)>}

nodes visited indices:  [1, 2, 3, 5]

Objective value:  4.0  Number of nodes visited:  4

nodes_visited by each driver:  [[(0, 2, 0), (2, 7, 0)], [(0, 3, 1), (3, 7, 1)], [(0, 1, 2), (1, 7, 2)], [], [(0, 5, 4), (5, 7, 4)], [], [], [], [], []]

time_visited:  {(0, 0): 1, (0, 1): 3, (0, 2): 5, (0, 3): 2, (0, 4): 1, (0, 5): 2, (0, 6): 3, (0, 7): 3, (0, 8): 3, (0, 9): 7, (2, 0): 1.223606797749979, (7, 0): 1.4472135954999579, (3, 1): 4.019803902718557, (7, 1): 5.03960